###Imports

In [3]:
!pip install Unidecode
!pip install hdbscan
import pickle
import unidecode
import hdbscan
import re
import nltk
from nltk import FreqDist
from nltk.util import ngrams
from nltk.tokenize import word_tokenize
import string
from string import punctuation
import pandas as pd
import numpy as np
import collections
from tqdm import tqdm
from gensim.models import Word2Vec, FastText
from sklearn.preprocessing import StandardScaler
from sklearn.metrics.pairwise import cosine_similarity, cosine_distances
import umap

###Funções

In [ ]:
def grava(obj, path, filename):
	pkl_file = open(path + filename, 'wb')
	pickle.dump(obj, pkl_file)
	pkl_file.close()

def abre(path, filename):
	pkl_file = open(path + filename, 'rb')
	obj = pickle.load(pkl_file)
	pkl_file.close()
	return obj

def get_nome_representacao_do_grupo_v4(df2,qtd_palavras,unidades):
	percentual_pra_manter_palavra_na_representacao = 0.50
	#pega cada palavra e ve as que mais se repetem nas sentences
	#fica com aquelas que estao em mais do que X% das sentences
	sentences = [sent for sent in df2['xprod']]
	if len(set(sentences)) == 1: #ou seja, todos os itens sao iguais.
		representacao_grupo = [word for word in set(sentences)][0].split()
	else:
		palavras_series = df2['xprod'].str.split()
		palavras_series = palavras_series.apply(lambda x: x[:qtd_palavras])
		contagem_palavras_nas_sentences = {}
		palav = [item for sublist in palavras_series for item in sublist]
		palavras = sorted( set(palav), key=palav.index) #pra preservar a ordem em que as palavras aparecem (set puro coloca em ordem alfabetica)
		for palavra in palavras:
			contagem_palavras_nas_sentences[palavra] = (palavras_series.apply(lambda x: palavra in x)).sum()
	
		contagem_palavras_nas_sentences = pd.Series(contagem_palavras_nas_sentences)
		contagem_palavras_nas_sentences = contagem_palavras_nas_sentences / len(df2)
		contagem_palavras_nas_sentences = contagem_palavras_nas_sentences[contagem_palavras_nas_sentences > percentual_pra_manter_palavra_na_representacao]
		representacao_grupo = list(contagem_palavras_nas_sentences.index)
	#reordenacao:
	primeiras_palavras = [word for word in representacao_grupo if ((not word.isdigit()) and word not in unidades)]
	meio_palavras = [word for word in representacao_grupo if word.isdigit()]
	ultimas_palavras = [word for word in representacao_grupo if word in unidades]
	
  # intercala numeros e unidades:
	if len(meio_palavras) == len(ultimas_palavras):
		result = [None]*(len(meio_palavras)+len(ultimas_palavras))
		result[::2] = meio_palavras
		result[1::2] = ultimas_palavras
		meio_palavras = result
		ultimas_palavras = []
	else:
		# if 'x' in representacao_grupo:
		if (('x' in ultimas_palavras) and (len(meio_palavras) > 0)):
			ultimas_palavras = [word for word in ultimas_palavras if word != 'x'] #retira o 'x', vai inserir abaixo:
			meio_palavras.insert(1,'x') #insere o 'x' apos o 1o numero
	if len(meio_palavras) == 0:
		representacao_grupo = primeiras_palavras #daih nao coloca unidades
	else:
		representacao_grupo = primeiras_palavras + meio_palavras + ultimas_palavras
	return representacao_grupo

def print_exemplos_grupos_v2_aleatorio(df,grupos,grupox,cols,qtd_palavras,unidades):
	#inicio = np.random.randint(len(grupos)-qtd_grupos_mostrar)
	#fim = inicio + qtd_grupos_mostrar
	for grupo in grupos:
		df_mostrar = df[df[grupox] == grupo]
		print('\nGrupo:',grupo,'len:',len(df_mostrar))
		print(get_nome_representacao_do_grupo_v4(df2=df[df[grupox]==grupo],qtd_palavras=qtd_palavras,unidades=unidades))
		print(df_mostrar[cols])

def calc_preco(df, grupos, grupox):
	cols = ['NCM','xprod', 'vuncom']
	df_list_grupos = []

	groups = []
	group_names = []
	media = []
	mediana = []
	media_saneada = []
	precos_max = []
	precos_min = []

	for i, grupo in enumerate(grupos):
		df_grupo = df[df[grupox] == grupo]
		df_list_grupos.append(df_grupo)	
		
		 #S = E – M sobrepreço estimado - referencial mercado
		media.append(round(df_grupo['vuncom'].mean(),3))
		mediana.append(round(df_grupo['vuncom'].median(),3))
		
		nova_media, preco_max, preco_min = calc_media_saneada(df_grupo)
		
		media_saneada.append(nova_media)
		precos_max.append(preco_max)
		precos_min.append(preco_min)
	
		descricao = get_nome_representacao_do_grupo_v4(df2=df[df[grupox]==grupo],qtd_palavras=qtd_palavras,unidades=unidades)
		descricao = ' '.join(descricao)
		group_names.append(descricao)

		groups.append(i+1)
	
	df_precos = pd.DataFrame(zip(groups, group_names, media, mediana, precos_max, precos_min, media_saneada), columns=["Grupo","Descricao", "Media", "Mediana", "Max", "Min", "Media Saneada"])
	
	return df_precos, df_list_grupos

def calc_media_saneada(df):
	preco = df['vuncom'].dropna()
	preco_max = max(preco)
	preco_min = min(preco)
	preco_mean = preco.mean()
	LS = preco_mean + preco.std()
	LI = preco_mean - preco.std()		
	CV = abs(preco.std() / preco_mean)
	
	if CV > 0.25:
		precos = preco[preco.between(LI, LS)]
		return (precos.mean(), preco_max, preco_min)
	else:
		return (preco_mean, preco_max, preco_min)

def descricoes_grupos(df, grupos, grupox):
	groups = []
	descrs = []
	for i, grupo in enumerate(grupos):
		df_grupo = df[df[grupox] == grupo]
		descr_unicas = df_grupo['xprod'].unique()
		for descr in descr_unicas:
			groups.append(i+1)
			descrs.append(descr)
	df_descrs_unicas = pd.DataFrame(zip(descrs, groups), columns=["Descricao", "Grupo"])
	return df_descrs_unicas

def save_groups(dfs):
	for i, df in enumerate(dfs):
		df.to_excel(f'grupo{i}.xlsx')


###Variaveis

In [ ]:
qtd_dimensoes = 300
qtd_dimensoes_umap = 10
qtd_palavras = 10
min_samples = 30
tamanho_minimo_pra_formar_grupo = 50
similarity_minima_pra_juntar_grupos = 0.58
quantile_a_retirar_outliers_dbscan = 0.95
percentual_primeira_palavra_igual_pra_considerar_grupo_homogeneo = 0.70
similarity_minima_pra_encaixar_itens_excluidos_no_final = 0.9

unidades = ['x','mm','m','cm','ml','g','mg','kg','unidade','unidades','polegada','polegadas','grama','gramas','gb','mb','l','lr','lt','litro','litros','mts','un','mgml','w','hz','v','gr','lts','lonas','cores','mcg']

path_drive = 'drive/My Drive/Residencia/'

###NCSMs 
- `Agua de beber`: 22019000 e 22011000
- 19053100 (biscoito), 20089900 (frutas e vegetais)
- `medicamentos`: 30049099 e 30049069 
- `outros`: 90211020 (parafusos), peças de veiculos (87089990), oleos de motor (27101932), juntas e tubos de borracha (39174090)


In [ ]:
df = pd.read_csv('notas.csv')
df_original = df.copy()
print(collections.Counter(df['ncm'].values.tolist()).most_common(10))

[(27101921, 12671), (27101259, 10675), (87089990, 8905), (30049099, 8335), (39174090, 7462), (19053100, 6437), (30049069, 6414), (27101932, 4419), (90211020, 4209), (20089900, 3840)]


###Meds


In [6]:
df_meds_notas = pd.read_csv('meds_notas.csv')
skipr= list(range(0,51))
df_meds = pd.read_excel('meds_2020_06_v3.xls', skiprows=skipr)
df_meds.columns = df_meds.columns.str.replace(' ','_')

In [7]:
df_meds.head()
df_meds_notas.head()
#df_med = df_med.drop(['day', 'poutcome'], axis=1)
groups = df_meds_notas.groupby(['med_cprodanvisa'])['ncm'].count().reset_index(name="Total")
groups['Media_Notas'] = df_meds_notas.groupby(['med_cprodanvisa']).mean()['vuncom'].values.tolist()
groups['Max_Notas'] = df_meds_notas.groupby(['med_cprodanvisa']).max()['vuncom'].values.tolist()
groups['Min_Notas'] = df_meds_notas.groupby(['med_cprodanvisa']).min()['vuncom'].values.tolist()

precos_gov = []
precos_gov_imposto = []
for cod in groups['med_cprodanvisa'].values.tolist():
  try:
    precos_gov.append(df_meds[df_meds['REGISTRO'] == int(cod)]['PMVG_Sem_Impostos'].values.tolist()[0])
  except:
    precos_gov.append(0)

  try:
    precos_gov_imposto.append(df_meds[df_meds['REGISTRO'] == int(cod)]['PMVG_18%'].values.tolist()[0])
  except:
    precos_gov_imposto.append(0)

groups['PMVG_Sem_Impostos'] = precos_gov
groups['PMVG_18%'] = precos_gov
groups[150:200]

,med_cprodanvisa,Total,Media_Notas,Max_Notas,Min_Notas,PMVG_Sem_Impostos,PMVG_18%
0,0000000000000,227,3.397344,104.3100,0.0300,0,0
1,0000000000001,77,1.520909,20.4300,0.0300,0,0
2,0000000000027,4,0.915000,2.2500,0.0500,0,0
3,0000000000199,23,1.334348,5.3000,0.0400,0,0
4,0000000002584,1,0.599000,0.5990,0.5990,0,0
5,0000000199206,27,2.482941,16.7983,0.0274,0,0
6,0000000272010,2,0.039650,0.0439,0.0354,0,0
7,0000001002351,15,33.042667,240.0000,0.6900,0,0
8,0000001003923,2,0.711100,0.7111,0.7111,0,0
9,0000001015201,16,46.691250,153.1300,1.1300,0,0


In [10]:
groups[1000:1050]

,med_cprodanvisa,Total,Media_Notas,Max_Notas,Min_Notas,PMVG_Sem_Impostos,PMVG_18%
1000,1029803740062,9,2.605556,8.76,0.02,0,0
1001,1029803770026,55,59.559091,197.01,6.04,"20,89","20,89"
1002,1029803840016,226,2.384336,8.00,0.13,0,0
1003,1029803900124,55,59.559091,197.01,6.04,"289,54","289,54"
1004,1029803920109,1,31.720000,31.72,31.72,"474,36","474,36"
1005,1029804040013,2,8.295000,11.99,4.60,"109,15","109,15"
1006,1029804040031,3,12.966667,19.00,0.90,"213,94","213,94"
1007,1029804310010,16,73.345000,408.00,1.10,0,0
1008,1029804310096,96,16.277709,231.25,0.05,"234,81","234,81"
1009,1029804460012,181,1.022541,13.00,0.08,0,0


In [11]:
df_meds_notas.head()

,ncm,med_cprodanvisa,xprod,vuncom
0,30049069,0000100410107,benzilpenicilina benzatina po p/ susp. inj. 60...,12.15
1,30049069,1108500100036,benzilpenicilina benzatina po p/ susp. inj. 60...,12.15
2,30049069,0000101390019,benzilpenicilina benzatina po p/ susp. inj. 60...,12.15
3,30049069,0000100410107,benzilpenicilina benzatina po p/ susp. inj. 60...,12.15
4,30049069,1108500010215,benzilpenicilina benzatina po p/ susp. inj. 60...,12.15


###Descriptions Cleaning

In [ ]:
df = df[df['ncm'].apply(lambda ncm: ncm == 22019000 or ncm == 22011000)]
print(len(df))

#df['xprod'].iloc[np.random.randint(len(df),size=50)]
#df['xprod'].iloc[np.random.randint(len(df),size=50)].apply(len)

3894


In [ ]:
nltk.download('stopwords')

pt_stopwords = set(nltk.corpus.stopwords.words("portuguese"))
stopwords = set(list(punctuation))

stopwords.add('potavel')
stopwords.add('garraf')
stopwords.add('garrafa')
stopwords.add('garrafao')
stopwords.add('botijao')
stopwords.add('garrafoes')
stopwords.add('natural')
stopwords.add('inamar')
stopwords.add('retornavel')
stopwords.add('gf')
stopwords.add('sterbom')
stopwords.add('vivara')
stopwords.add('potiguar')
stopwords.add('san')
stopwords.add('vale')
stopwords.add('ville')
stopwords.add('gotas')
stopwords.add('cristal')
stopwords.add('ster')
stopwords.add('bom')
stopwords.add('cristalina')
stopwords.add('natal')
stopwords.add('maria')
stopwords.add('santa')
stopwords.add('sta')
stopwords.add('cruz')
stopwords.add('comarcas')
stopwords.add('liquido')
stopwords.add('envase')
stopwords.add('envasado')
stopwords.add('envasada')
stopwords.add('tj')
stopwords.add('pp')

pt_stopwords.update(stopwords)

#limpa sentencas retirando stopwords, pontuacao e deixa minusculo.
df['xprod'] = [ ' '.join([word.lower() for word in descr.split() if word.lower() not in pt_stopwords]) for descr in df['xprod'].astype(str)]
#insere espaco apos / e -, pra no final nao ficar palavras assim: csolucao, ptexto (originais eram c/solucao, p-texto)
df['xprod'] = df['xprod'].apply(lambda descr: re.sub(r'/|-',r' ',descr))
#retira pontuacao, /, etc:
df['xprod'] = df['xprod'].apply(lambda x: x.translate(str.maketrans('', '', string.punctuation)))
#insere espaco apos numero e letra (separa unidades de medida:) ex.: 500ml vs 100ml vs 500mg
df['xprod'] = df['xprod'].apply(lambda x: re.sub(r'(\d{1})(\D)',r'\1 \2',x))
#insere espaco apos letra e numero ex.:c100 pc50
df['xprod'] = df['xprod'].apply(lambda x: re.sub(r'(\D{1})(\d)',r'\1 \2',x))
#apaga caracteres de tamanho 1
df['xprod'] = df['xprod'].apply(lambda x: re.sub(r'\b[a-zA-Z]\b',r'',x))
#retira espacos duplicados
df['xprod'] = df['xprod'].apply(lambda x: re.sub(r' +',r' ', x))
#padronizar agua mineral
df['xprod'] = df['xprod'].apply(lambda x: re.sub(r' min ',r' mineral ', x))
#retira espaco no inicio da frase
df['xprod'] = df['xprod'].apply(lambda x: x.strip())
#retira acentos:
df['xprod'] = df['xprod'].apply(lambda x: unidecode.unidecode(x))
# remove zeros a esquerda de numeros (02 litros, 05, etc.)
df['xprod'] = df['xprod'].apply(lambda x: ' '.join([word.lstrip('0') for word in x.split()] ) )
# remove 'x', pra não diferenciar pneu 275 80 de 275 x 80:
df['xprod'] = df['xprod'].apply(lambda x: ' '.join([word for word in x.split() if word is not 'x']))
#se primeira palavra for numero apaga 
df['xprod'] = df['xprod'].apply(lambda x: ' '.join(x.split()[1:]) if ((len(x) > 1) and (x.split()[0].isdigit()) ) else x)
#muda l e lts e litros por lt
df['xprod'] = df['xprod'].apply(lambda x: re.sub(r' l ',r'lt', x))
df['xprod'] = df['xprod'].apply(lambda x: re.sub(r'lts',r'lt', x))
df['xprod'] = df['xprod'].apply(lambda x: re.sub(r'litros',r'lt', x))
df['xprod'] = df['xprod'].apply(lambda x: re.sub(r'litro',r'lt', x))
#muda l e lts por lt se for no final de frase
df['xprod'] = df['xprod'].apply(lambda x: x  if x.split()[-1] != 'l' else re.sub(r' l',r' lt', x))
df['xprod'] = df['xprod'].apply(lambda x: x  if x.split()[-1] != 'lts' else re.sub(r'lts',r'lt', x))
df['xprod'] = df['xprod'].apply(lambda x: x  if x.split()[-1] != 'litro' else re.sub(r'litro',r'lt', x))
df['xprod'] = df['xprod'].apply(lambda x: x  if x.split()[-1] != 'litros' else re.sub(r'litros',r'lt', x))
#substitui 1000ml por 1l 
df['xprod'] = df['xprod'].apply(lambda x: re.sub(r'1000 ml',r'1 lt',x))


#limpa palavras novamente
df['xprod'] = [ ' '.join([word for word in descr.split() if word.lower() not in pt_stopwords]) for descr in df['xprod'].astype(str)]

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
#df['xprod'].iloc[np.random.randint(len(df),size=50)]

###Pre processing

In [ ]:
sentences = [descr.split()[:qtd_palavras] for descr in df['xprod']]

'''
model = FastText(size=qtd_dimensoes, min_count=1)  
model.build_vocab(sentences=sentences)
model.train(sentences=sentences, total_examples=len(sentences), epochs=10)
'''
model2 = Word2Vec(sentences,size=qtd_dimensoes, min_count=1, workers=-1)

####Aplicando pesos

#####Fasttext

In [ ]:
'''
#Conversao word embeddings to sentence embedding, com pesos
#dando peso maior pras primeiras palavras, peso decrescente ateh o final, numeros com mesmo peso da primeira palavra:
doc_vectors = {}
for number, sent in enumerate(tqdm(sentences)):
    if len(sent) == 0:
        doc_vectors[number] = np.zeros(qtd_dimensoes,)
    elif len(sent) == 1:
        doc_vectors[number] = model[sent[0]]
    elif len(sent) > 1:
        pesos = np.array(range(len(sent))[::]) + 1
        # agora com pesos 1/x - tem que ser na ordem 1,2,..., os menores numeros dao maiores pesos
        pesos = 1 / pesos 
        media = []
        divisao = 0
        counter = 0
        for word in sent:
            if word.isdigit():
                #se eh digit, atribui peso NO 3/4 da faixa entre o peso da primeira e da ultima palavra. 
                #Mesmo peso pra todos os numeros, mais importante que palavras do fim, menos importante que palavras do inicio.
                media.append(model.wv[word] * ((pesos[0]+pesos[-1])*(1/4))) 
                divisao += ((pesos[0]+pesos[-1])*(1/4))
            else:
                media.append(model.wv[word] * pesos[counter])
                divisao += pesos[counter]
            counter += 1
        doc_vectors[number] = np.array(media).sum(axis=0) / divisao #media de tudo
'''

'\n#Conversao word embeddings to sentence embedding, com pesos\n#dando peso maior pras primeiras palavras, peso decrescente ateh o final, numeros com mesmo peso da primeira palavra:\ndoc_vectors = {}\nfor number, sent in enumerate(tqdm(sentences)):\n    if len(sent) == 0:\n        doc_vectors[number] = np.zeros(qtd_dimensoes,)\n    elif len(sent) == 1:\n        doc_vectors[number] = model[sent[0]]\n    elif len(sent) > 1:\n        pesos = np.array(range(len(sent))[::]) + 1\n        # agora com pesos 1/x - tem que ser na ordem 1,2,..., os menores numeros dao maiores pesos\n        pesos = 1 / pesos \n        media = []\n        divisao = 0\n        counter = 0\n        for word in sent:\n            if word.isdigit():\n                #se eh digit, atribui peso NO 3/4 da faixa entre o peso da primeira e da ultima palavra. \n                #Mesmo peso pra todos os numeros, mais importante que palavras do fim, menos importante que palavras do inicio.\n                media.append(model.w

#####Word2vec

In [ ]:
doc_vectors2 = {}

for number, sent in enumerate(tqdm(sentences)):
    #agora dando peso maior pras primeiras palavras, peso decrescente ateh o final, numeros com mesmo peso da primeira palavra:
    if len(sent) == 0:
        doc_vectors2[number] = np.zeros(qtd_dimensoes,)
    elif len(sent) == 1:
        doc_vectors2[number] = model2.wv[sent[0]]
    elif len(sent) > 1:
        pesos = np.array(range(len(sent))[::]) + 1
        pesos = 1 / pesos
        media = []
        divisao = 0
        counter = 0
        for word in sent:
            #### AGORA O MODEL EH W2V E O PESO EH DOBRADO PRA DIGITS:
            if word.isdigit():
                media.append(model2.wv[word] * ((pesos[0]+pesos[-1])*(1/2)) )
                divisao += ((pesos[0]+pesos[-1])*(1/2))
            else:
                media.append(model2.wv[word] * pesos[counter])
                divisao += pesos[counter]
            counter += 1
        doc_vectors2[number] = np.array(media).sum(axis=0) / divisao #media de tudo

100%|██████████| 3894/3894 [00:00<00:00, 23566.33it/s]


In [ ]:
doc_vectors2 = pd.DataFrame(doc_vectors2).T
doc_vectors3 = doc_vectors2
doc_vectors2 = doc_vectors2.set_index(df.index) 

scaler = StandardScaler()
doc_vectors_std_df = pd.DataFrame(scaler.fit_transform(doc_vectors2),index=doc_vectors2.index,columns=doc_vectors2.columns)

####Redução de dimensão

In [ ]:
#n_neighbors=20
umap_redux = umap.UMAP(n_components=qtd_dimensoes_umap, random_state=999, metric='cosine')
umap_redux.fit(doc_vectors_std_df)

doc_vectors_std_df_umap = umap_redux.transform(X=doc_vectors_std_df)

###Clustering

In [ ]:
clustering = hdbscan.HDBSCAN(min_cluster_size=tamanho_minimo_pra_formar_grupo,min_samples=min_samples,prediction_data=True,core_dist_n_jobs=-1).fit(doc_vectors_std_df_umap)
#clustering = abre(path_drive, 'HDBSCAN_modelo.pkl')

df['grupo'] = clustering.labels_

In [ ]:
threshold = pd.Series(clustering.outlier_scores_).quantile(quantile_a_retirar_outliers_dbscan)
outliers = np.where(clustering.outlier_scores_ > threshold)[0]
df.iloc[outliers,df.columns.get_loc('grupo')] = -2

In [ ]:
grupos = np.unique(clustering.labels_)
grupos = [grupo for grupo in grupos if grupo >= 0]

print('qtd de grupos:',len(grupos))

qtd de grupos: 16


###Cluster Cleaning

In [ ]:
#########################################
# EXCLUI sentences outliers pelo texto pelos exemplars
# FAZ cosine distance pra cada sentence dentro dos grupos
#########################################

#pontos mais representativos de cada grupo (como hdbscan aceita formato aleatorio de cada cluster, nao tem como usar centroide)
exemplars = []
for exemplar in tqdm(clustering.exemplars_):
    exemplars.append(np.mean(exemplar,axis=0))
exemplars_df = pd.DataFrame(exemplars,index=range(len(grupos)))

map_grupos_exemplars = {}
df_temp = pd.DataFrame(columns=['sims'])

for grupo in tqdm(grupos[:]):
    df2 = df[df['grupo'] == grupo]
    indexes = df2.index
    grupo_vectors = pd.DataFrame(doc_vectors_std_df_umap,index=df.index).loc[indexes]
    
    grupo_do_exemplar = pd.Series(cosine_similarity(grupo_vectors.mean(axis=0).values.reshape(1,-1),exemplars_df)[0]).sort_values(ascending=False).index[0]
    map_grupos_exemplars[grupo] = grupo_do_exemplar

    sims = cosine_similarity(grupo_vectors,exemplars[grupo_do_exemplar].reshape(1,-1))

    df2['sims'] = sims
    df_temp = df_temp.append(df2[['sims']])

#passa resultados pro df principal:
df['sims'] = df_temp
df['sims'] = df['sims'].replace(np.nan,-1)

#retira quem tem similaridade negativa - eh um bom parametro.
df['grupo2'] = np.where(df['sims'] < 0, -1, df['grupo'])

grupos = df['grupo2'].unique()
grupos = [grupo for grupo in grupos if grupo >= 0]

print('qtd de grupos:',len(grupos))

  0%|          | 0/16 [00:00<?, ?it/s]/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http

qtd de grupos: 16


In [ ]:
##########################################
# Se a 1a palavra nao for a mesma em X% do grupo, exclui o grupo, eh muito heterogeneo
##########################################
df['grupo3'] = df['grupo2']

grupos = sorted(df['grupo3'].unique())
grupos = [grupo for grupo in grupos if grupo >=0] #tirar os -1, -2, etc.

grupos_homogeneos = []

for grupo in tqdm(grupos):
    df2 = df[df['grupo3'] == grupo]
    if len(df2) > 0:
        if ( df2['xprod'].apply(lambda x: x.split()[0] if (len(x.split()) > 0) else np.random.random()).value_counts().iloc[0] / len(df2) ) > percentual_primeira_palavra_igual_pra_considerar_grupo_homogeneo:
            grupos_homogeneos.append(grupo)

df['grupo4'] = df['grupo3'].isin(grupos_homogeneos)
df['grupo4'] = np.where(df['grupo4'], df['grupo3'], -1)

grupos = sorted(df['grupo4'].unique())
grupos = [grupo for grupo in grupos if grupo >=0] 

print('qtd de grupos:',len(grupos))

100%|██████████| 16/16 [00:00<00:00, 433.01it/s]


qtd de grupos: 15


In [ ]:
############################################################################
# EXCLUI GRUPOS NAO HOMOGENEOS - pela contagem de palavras diferentes! 
############################################################################
quantile_a_retirar_quantidade_palavras_diferentes_no_grupo = 0.95

df['qtd_palavras_diferentes'] = df['xprod'].apply(lambda x: len(set( [item for sublist in [sent.split()[:qtd_palavras] for sent in x] for item in sublist] )))
qtd_palavras_por_grupo = df.groupby('grupo4')['qtd_palavras_diferentes'].median()
qtd_palavras_por_grupo = qtd_palavras_por_grupo.sort_values()
qtd_max_palavras_diferentes_no_grupo = int(qtd_palavras_por_grupo.quantile(quantile_a_retirar_quantidade_palavras_diferentes_no_grupo))
print('Quantidade maxima de palavras diferentes aceita por grupo:', qtd_max_palavras_diferentes_no_grupo)

df['qtd_median_palavras_dif_grupo'] = df['grupo4'].map(qtd_palavras_por_grupo)

df['grupo5'] = np.where(df['qtd_median_palavras_dif_grupo'] > qtd_max_palavras_diferentes_no_grupo, -1, df['grupo4'])

grupos = df['grupo5'].unique()
grupos = [grupo for grupo in grupos if grupo >=0]

print('qtd de grupos:',len(grupos))

Quantidade maxima de palavras diferentes aceita por grupo: 14
qtd de grupos: 14


In [ ]:
#pulei passo do grupo6
df['grupo6'] = df['grupo5']
#df['grupo6'] = df['grupo4']
############################################################################
# EXCLUI GRUPOS que tem MUITOS NUMEROS DIFERENTES NAS SENTENCES
############################################################################
quantile_a_retirar_numeros_diferentes_no_grupo = 0.95

df['qtd_numeros_diferentes'] = df['xprod'].apply(lambda x: len(set( [item for sublist in [sent.split()[:qtd_palavras] for sent in x] for item in sublist if item.isdigit()   ] ) ))
df['qtd_numeros_diferentes'] = np.where(df['qtd_numeros_diferentes'] == 0, 0, df['qtd_numeros_diferentes']-1)
qtd_numeros_por_grupo = df.groupby('grupo4')['qtd_numeros_diferentes'].median()
qtd_numeros_por_grupo = qtd_numeros_por_grupo[qtd_numeros_por_grupo > 0]
qtd_numeros_por_grupo = qtd_numeros_por_grupo.sort_values()
qtd_max_numeros_diferentes_no_grupo = int(qtd_numeros_por_grupo.quantile(quantile_a_retirar_numeros_diferentes_no_grupo))
print('Quantidade maxima de numeros diferentes aceita por grupo:', qtd_max_numeros_diferentes_no_grupo)

df['qtd_median_numeros_dif_grupo'] = df['grupo4'].map(qtd_numeros_por_grupo)

df['grupo7'] = np.where(df['qtd_median_numeros_dif_grupo'] > qtd_max_numeros_diferentes_no_grupo, -1, df['grupo6'])

grupos = df['grupo7'].unique()
grupos = [grupo for grupo in grupos if grupo >=0] 

print('qtd de grupos:',len(grupos))

Quantidade maxima de numeros diferentes aceita por grupo: 1
qtd de grupos: 13


In [ ]:
doc_vectors3 = doc_vectors3.set_index(df.index) 
doc_vectors_grupos = {}

for grupo in tqdm(grupos):
    indices = df[df['grupo7'] == grupo].index
    doc_vectors_grupos[grupo] = doc_vectors3.loc[indices]
    doc_vectors_grupos[grupo] = doc_vectors_grupos[grupo].mean(axis=0)   

doc_vectors_grupos = pd.DataFrame(doc_vectors_grupos).T
#usa o scaler original:
doc_vectors_grupos_std = pd.DataFrame(scaler.transform(doc_vectors_grupos),index=doc_vectors_grupos.index,columns=doc_vectors_grupos.columns)

grupos_similarities = cosine_similarity(doc_vectors_grupos_std)
grupos_similarities = pd.DataFrame(grupos_similarities,index=doc_vectors_grupos.index,columns=doc_vectors_grupos.index)

#junta os grupos:
grupos_similares = []
for grupo in tqdm(grupos_similarities):
    agrupar_df = grupos_similarities[grupo].sort_values(ascending=False)
    agrupar_df = agrupar_df[agrupar_df >= similarity_minima_pra_juntar_grupos]
    grupos_similares.append(list(agrupar_df.index))

100%|██████████| 13/13 [00:00<00:00, 813.81it/s]


In [ ]:
novo_grupo = 0
mapeamento_grupos = {}
for mini_grupo in tqdm(grupos_similares):
    if len(mini_grupo) == 1:
        mapeamento_grupos[mini_grupo[0]] = novo_grupo
    else:
        for grupo in mini_grupo:
            if grupo not in mapeamento_grupos.keys():
                for mini_grupo2 in grupos_similares:
                    if grupo in mini_grupo2:
                        mapeamento_grupos[grupo] = novo_grupo
                        for grupo2 in mini_grupo2:
                            if grupo2 not in mapeamento_grupos.keys():
                                mapeamento_grupos[grupo2] = novo_grupo
    novo_grupo += 1

df['grupo9'] = df['grupo7'].map(mapeamento_grupos)
df['grupo9'] = df['grupo9'].fillna(-1)
df['grupo9'] = df['grupo9'].astype(int)

grupos = df['grupo9'].unique()
grupos = [grupo for grupo in grupos if grupo >=0] 

print('qtd de grupos:',len(grupos))

100%|██████████| 13/13 [00:00<00:00, 6260.87it/s]

qtd de grupos: 10


In [ ]:
###############################################
# Repassar as sentences do grupo -1 e tentar encaixá-las nos grupos formados, só com grande certeza.
###############################################

excluidos_index =  df[df['grupo9'] == -1].index
incluidos_index =  df[df['grupo9'] >= 0].index

doc_vectors_excluidos = doc_vectors2.loc[excluidos_index]
doc_vectors_incluidos = doc_vectors2.loc[incluidos_index]

doc_vectors_grupos_finais = {}
for grupo in tqdm(grupos):
    df2 = df[df['grupo9'] == grupo]
    doc_vectors_grupos_finais[grupo] = doc_vectors_incluidos.loc[df2.index]
    doc_vectors_grupos_finais[grupo] = doc_vectors_grupos_finais[grupo].values.mean(axis=0)

# index aqui jah eh o numero certo dos grupos:
doc_vectors_grupos_finais = pd.DataFrame(doc_vectors_grupos_finais).T
compara = cosine_similarity(doc_vectors_excluidos.loc[excluidos_index],doc_vectors_grupos_finais.values)
compara = pd.DataFrame(compara,index=excluidos_index, columns=grupos)
similarity_do_grupo_mais_parecido = compara.max(axis=1)
grupo_mais_parecido = compara.idxmax(axis=1)

encaixar_excluidos = pd.Series( np.where(similarity_do_grupo_mais_parecido >= similarity_minima_pra_encaixar_itens_excluidos_no_final, grupo_mais_parecido, -1), index= similarity_do_grupo_mais_parecido.index)
df['grupo10'] = encaixar_excluidos
df['grupo10'] = df['grupo10'].fillna(-1)
df['grupo10'] = np.where(df['grupo10'] == -1, df['grupo9'], df['grupo10'])

grupos = df['grupo10'].unique()
grupos = [grupo for grupo in grupos if grupo >=0] 

print('% de sentencas uteis:', (df['grupo10']>=0).sum() / len(df) )
print('qtd de grupos:',len(grupos))


100%|██████████| 10/10 [00:00<00:00, 680.21it/s]

% de sentencas uteis: 0.8949666153055984
qtd de grupos: 10


###Resultados

In [ ]:
#print_exemplos_grupos_v2_aleatorio(df=df,grupos=grupos,grupox='grupo7',cols=['xprod', 'vuncom'],qtd_palavras=qtd_palavras,unidades=unidades)

In [ ]:
#grupos_similares
grupos_similarities

,0,11,6,8,13,14,2,10,1,5,4,7,3
0,0.999999,-0.384637,-0.276973,-0.355350,-0.368712,-0.315621,0.425096,-0.425877,-0.169379,-0.578910,-0.003765,-0.214644,-0.387381
11,-0.384637,1.000000,0.248829,0.025511,0.335191,0.272328,-0.496595,0.021959,0.069747,0.200525,-0.327862,0.037724,0.303412
6,-0.276973,0.248829,1.000000,-0.188956,0.877102,0.142629,-0.386530,0.001290,0.293130,0.122178,-0.506950,-0.142145,0.173764
8,-0.355350,0.025511,-0.188956,1.000000,-0.030057,0.278870,-0.263140,-0.097508,-0.564309,0.725703,0.313141,0.939764,0.087892
13,-0.368712,0.335191,0.877102,-0.030057,1.000000,0.348390,-0.546565,-0.176511,0.114797,0.206544,-0.593176,0.019209,0.269323
14,-0.315621,0.272328,0.142629,0.278870,0.348390,1.000000,-0.460899,-0.373351,-0.285014,0.225297,-0.337862,0.287765,0.164542
2,0.425096,-0.496595,-0.386530,-0.263140,-0.546565,-0.460899,1.000000,-0.203744,-0.043764,-0.583224,0.407948,-0.395891,-0.598387
10,-0.425877,0.021959,0.001290,-0.097508,-0.176511,-0.373351,-0.203744,1.000000,0.548588,0.294627,0.310167,-0.138046,0.279822
1,-0.169379,0.069747,0.293130,-0.564309,0.114797,-0.285014,-0.043764,0.548588,1.000000,0.068988,-0.478826,-0.558762,0.074033
5,-0.578910,0.200525,0.122178,0.725703,0.206544,0.225297,-0.583224,0.294627,0.068988,1.000000,-0.112380,0.764884,0.309465


In [ ]:
print_exemplos_grupos_v2_aleatorio(df=df,grupos=grupos,grupox='grupo10',cols=['xprod', 'vuncom'],qtd_palavras=qtd_palavras,unidades=unidades)


Grupo: 0.0 len: 1184
['agua', 'mineral', '20', 'lt']
                     xprod  vuncom
654     agua mineral 20 lt    4.45
1201    agua mineral 20 lt    6.30
1321    agua mineral 20 lt    6.30
1327    agua mineral 20 lt    3.25
1545    agua mineral 20 lt    6.30
...                    ...     ...
482002  agua mineral 20 lt    5.65
482144  agua mineral 20 lt    5.40
482145  agua mineral 20 lt    5.40
482146  agua mineral 20 lt    5.40
482147  agua mineral 20 lt    5.40

[1184 rows x 2 columns]

Grupo: 1.0 len: 68
['agua', 'mineral', '200', 'ml']
                           xprod  vuncom
657     agua mineral copo 200 ml   24.88
3409     agua mineral gas 200 ml   19.22
22922   agua mineral copo 200 ml    0.60
39256     agua mineral cp 200 ml   29.00
42562        agua mineral 200 ml    0.95
...                          ...     ...
451829            agua cp 200 ml   15.65
462098       agua mineral 200 ml    0.50
464172            agua cp 200 ml   15.65
469596  agua mineral copo 200 ml    0.

In [ ]:
values = [-1, -2] #verificar os que ficaram de fora (-1) e foram considerados outliers (-2)
out_inicio = df[df.grupo.isin(values)]

print(f'Ficaram fora no início: {len(out_inicio)}')

out_fim = df[df['grupo10']==-1]
print(f'Ficaram fora no fim: {len(out_fim)}')

qtd_produtos = [int(qtd) for ncm, qtd in collections.Counter(df['ncm'].values.tolist()).most_common(2)]

print(f'Total de registros de fora (%): {(len(out_inicio)+len(out_fim))/sum(qtd_produtos)*100}')

Ficaram fora no início: 631
Ficaram fora no fim: 409
Total de registros de fora (%): 26.70775552131484


In [ ]:
out_inicio.to_excel('ficaram_fora_inicio.xlsx')
out_fim.to_excel('ficaram_fora_fim.xlsx')

In [ ]:
df_precos, df_list_grupos = calc_preco(df=df,grupos=grupos,grupox='grupo10')
save_groups(df_list_grupos)
df_precos

,Grupo,Descricao,Media,Mediana,Max,Min,Media Saneada
0,1,agua mineral 20 lt,5.296,5.00,29.95,1.50,4.989288
1,2,agua mineral 200 ml,10.799,13.99,29.20,0.29,14.196970
2,3,agua mineral 500 ml,3.873,1.24,17.60,0.62,1.655202
3,4,agua adicionada sais 20,4.555,4.52,6.60,1.80,4.554529
4,5,agua mineral 20,5.361,5.50,9.00,1.50,5.536167
5,6,gelo,5.515,5.80,46.88,2.00,4.897313
6,7,agua 20,4.070,4.00,20.00,1.80,3.858962
7,8,agua,6.130,6.00,130.00,0.07,4.297377
8,9,agua mineral,5.348,5.50,7.00,0.95,5.347883
9,10,agua cx mil,30.882,30.00,33.00,30.00,30.882353


In [ ]:
df_descrs_unicas = descricoes_grupos(df=df,grupos=grupos,grupox='grupo10')
df_descrs_unicas

,Descricao,Grupo
0,agua mineral 20 lt,1
1,agua mineral oeste 20 lt und,1
2,agua mineral boutijao 20 lt,1
3,agua mineral gas acondicionada polipropileno c...,1
4,agua mineral 20 lt rio grande,1
...,...,...
136,agua mineral un,9
137,agua mineral 1500 lt,9
138,agua mineral ceu,9
139,agua mineral 202 lt agua,9


In [ ]:
df_original['grupo10'] = df['grupo10']
df_original['grupo10'] = df_original['grupo10'].fillna(-1)

df_descrs_unicas = descricoes_grupos(df=df_original,grupos=grupos,grupox='grupo10')
df_descrs_unicas

,Descricao,Grupo
0,agua mineral garrafão de 20 litros,1
1,água mineral de 20 litros garrafão ( inamar),1
2,agua mineral potiguar 20 lts ( liquido ),1
3,agua mineral 20 lts,1
4,agua mineral - 20 litros,1
...,...,...
278,agua mineral vivara,9
279,agua mineral santa cruz,9
280,agua mineral 202 litros - só água,9
281,agua potavel cx um m - mil litros,10


In [ ]:
grava(clustering, path_drive, 'HDBSCAN_modelo_agua.pkl')

###Predição